# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

import azureml.core
from azureml.core import Experiment, Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A84Z23R67 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-134046
aml-quickstarts-134046
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config() # this automatically looks for a directory .azureml

# Choose a name for your CPU cluster
cpu_cluster_name = "cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4, 
                                                            idle_seconds_before_scaledown=2400,
                                                            vm_priority='lowpriority')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
notebook134046 ComputeInstance Succeeded
cluster AmlCompute Succeeded


In [38]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '48a74bb7-9950-4cc1-9caa-5d50f995cc55'
resource_group = 'aml-quickstarts-134046'
workspace_name = 'quick-starts-ws-134046'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='dibetes')
df=dataset.to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [18]:
from azureml.train.automl import AutoMLConfig


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    debug_log = 'automl_errors.log',
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='Outcome',
    n_cross_validations=5,
    compute_target="cluster",
    iterations=24,
    max_concurrent_iterations=8)

In [20]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

Running on remote.
Running on remote compute: cluster
Parent Run ID: AutoML_25f8d913-9778-4444-a10c-8acd597ad2b9

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [21]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [22]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_25f8d913-9778-4444-a10c-8acd597ad2b9',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-09T05:02:53.277531Z',
 'endTimeUtc': '2021-01-09T05:26:45.361481Z',
 'properties': {'num_iterations': '24',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"diabetes","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-134046","workspace_name":"quick-starts-ws-134046","region":"southcentralus","compute_target":"cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":24,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto",

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [23]:
bestrun,model=remote_run.get_output()

In [24]:
#TODO: Save the best model
bestrun

Experiment,Id,Type,Status,Details Page,Docs Page
diabetes,AutoML_25f8d913-9778-4444-a10c-8acd597ad2b9_22,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                        degree=3,
                                                                                        gamma='scale',
                                                                                        kernel='rbf',
                                                                                        m

In [26]:
best_run_metrics = bestrun.get_metrics()

In [27]:
model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                        degree=3,
                                                                                        gamma='scale',
                                                                                        kernel='rbf',
                                                                                        m

In [28]:
import joblib
os.makedirs('./outputs', exist_ok=True)

joblib.dump(model, filename='outputs/automl_model.joblib')

model_name = bestrun.properties['model_name']
model_name

'AutoML25f8d913922'

In [29]:
env = bestrun.get_environment().save_to_directory(path='environments')

script_file = 'score.py'

bestrun.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

In [30]:
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML model')
model.id

'AutoML25f8d913922:1'

In [31]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'automl-diabetes'
print(aci_service_name)

automl-diabetes


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
State: Healthy
Scoring URI: http://a777970f-63ef-40af-a16f-0466710559e4.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [36]:
%run endpoint.py

{"result": [1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [39]:
df_test=df[700:710]

In [40]:
y_true=df_test.pop('Outcome')

In [41]:
sample_json = json.dumps({'data':df_test.to_dict(orient='records')})

In [42]:
sample_json

'{"data": [{"Pregnancies": 2, "Glucose": 122, "BloodPressure": 76, "SkinThickness": 27, "Insulin": 200, "BMI": 35.9, "DiabetesPedigreeFunction": 0.483, "Age": 26}, {"Pregnancies": 6, "Glucose": 125, "BloodPressure": 78, "SkinThickness": 31, "Insulin": 0, "BMI": 27.6, "DiabetesPedigreeFunction": 0.565, "Age": 49}, {"Pregnancies": 1, "Glucose": 168, "BloodPressure": 88, "SkinThickness": 29, "Insulin": 0, "BMI": 35.0, "DiabetesPedigreeFunction": 0.905, "Age": 52}, {"Pregnancies": 2, "Glucose": 129, "BloodPressure": 0, "SkinThickness": 0, "Insulin": 0, "BMI": 38.5, "DiabetesPedigreeFunction": 0.304, "Age": 41}, {"Pregnancies": 4, "Glucose": 110, "BloodPressure": 76, "SkinThickness": 20, "Insulin": 100, "BMI": 28.4, "DiabetesPedigreeFunction": 0.118, "Age": 27}, {"Pregnancies": 6, "Glucose": 80, "BloodPressure": 80, "SkinThickness": 36, "Insulin": 0, "BMI": 39.8, "DiabetesPedigreeFunction": 0.177, "Age": 28}, {"Pregnancies": 10, "Glucose": 115, "BloodPressure": 0, "SkinThickness": 0, "Insul

In [43]:
output = service.run(sample_json)

In [44]:
output

'{"result": [0, 0, 1, 1, 0, 0, 0, 0, 1, 0]}'

In [45]:
print("Predicted Values are :", output)
print("True Values are:",y_true.values)

Predicted Values are : {"result": [0, 0, 1, 1, 0, 0, 0, 0, 1, 0]}
True Values are: [0 1 1 0 0 0 1 0 1 1]


In [46]:
service.get_logs()

'2021-01-09T05:37:35,084144688+00:00 - iot-server/run \n2021-01-09T05:37:35,084144688+00:00 - gunicorn/run \n2021-01-09T05:37:35,085022646+00:00 - rsyslog/run \n2021-01-09T05:37:35,112180441+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [47]:
service.delete()